# 2040 le cap des 100% de voitures électriques 
*Etude data - Projet 8 - @Nalron (août 2020)*\
*Traitement des données sur Jupyter Notebook (Distribution Anaconda)*\
*Etude réalisée en langage Python*

Visualisation des Tableaux de bord: [Tableau Public](https://public.tableau.com/profile/nalron#!/vizhome/ElectricCarsFrance2040/Vuedensemble)

---

# Rappel des missions


### [Mission 1 : Positionnement de la voiture électrique en France](https://github.com/nalron/project_electric_cars_france2040/blob/french_version/p8_notebook01.ipynb)
Évolution du parc automobile électrique à 2 ans.<br>
Identification et classification des inégalités locales des voitures électriques.<br>
Autonomie et consommation moyenne d'une voiture électrique.

### [Mission 2 : Besoin des déploiements en IRVE](https://github.com/nalron/project_electric_cars_france2040/blob/french_version/p8_notebook02.ipynb)
Évolution du nombre de points de recharge disponibles ouverts au public.<br>
Analyse de la répartition par borne de recharge, type de prise et catégorie d’aménageur.<br>
Utilisation des ratios pour le dimensionnement d'un maillage de taille optimale.<br>
Prévision du nombre de PDC à horizon 2025.<br>

### [Mission 3 : Appel de charge au réseau électrique](https://github.com/nalron/project_electric_cars_france2040/blob/french_version/p8_notebook03.ipynb)
Analyse de la consommation d'électricité en France et des filières de production.<br>
Profiler un pic d’utilisation des bornes de recharge.<br>
Courbe de charge réseau électrique pour répondre aux nouveaux modes de consommation.

---

In [3]:
#Import des principales librairies Python
import pandas as pd
import plotly.figure_factory as ff
import requests
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Mission 3 : Appel de charge au réseau électrique<a id="reseau">

__`Traitement des données du parc de production d'énergie par typologie`__

In [217]:
#Chargement du jeu de données "parc-des-installations-de-production-raccordees-par-departement"
parc = pd.read_csv('p8_data/parc-des-installations-de-production-raccordees-par-departement.csv', sep=';')
parc

,Code Région,Région,Code Département,Nom Département,Code Type Prod,Type Production,ID_type_inj,Type Injection,Tranche,Fin De Trimestre,Nombre Installations,Puissance MW,Geo Shape,Geo Point
0,75,Nouvelle Aquitaine,33,Gironde,100,Photovoltaïque,1,Injection totale,2020-06-30,b-]36;100],143.0,11.16039,NaN,NaN
1,11,Île-de-France,75,Paris,100,Photovoltaïque,1,Injection totale,2020-06-30,b-]36;100],19.0,1.31682,NaN,NaN
2,32,Hauts-de-France,62,Pas-de-Calais,100,Photovoltaïque,2,Injection du surplus,2020-06-30,b-]36;100],7.0,0.50400,NaN,NaN
3,28,Normandie,76,Seine-Maritime,100,Photovoltaïque,3,Autoconsommation sans injection,2020-06-30,a-]0;36],143.0,0.48842,NaN,NaN
4,44,Grand-Est,52,Haute-Marne,100,Photovoltaïque,1,Injection totale,2020-06-30,a-]0;36],1032.0,4.43685,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,53,Bretagne,22,Côtes-d'Armor,600,Autres,1,Injection totale,2020-06-30,d-]250;...[,3.0,12.72400,NaN,NaN
1733,75,Nouvelle Aquitaine,40,Landes,600,Autres,2,Injection du surplus,2020-06-30,d-]250;...[,1.0,6.00000,NaN,NaN
1734,75,Nouvelle Aquitaine,47,Lot-et-Garonne,600,Autres,1,Injection totale,2020-06-30,d-]250;...[,1.0,4.99000,NaN,NaN
1735,84,Auvergne-Rhône-Alpes,1,Ain,600,Autres,1,Injection totale,2020-06-30,d-]250;...[,1.0,6.65000,NaN,NaN


In [218]:
#Liste des colonnes du jeu de données 
parc.columns

Index(['Code Région', 'Région', 'Code Département', 'Nom Département',
       'Code Type Prod', 'Type Production', 'ID_type_inj', 'Type Injection',
       'Tranche', 'Fin De Trimestre', 'Nombre Installations', 'Puissance MW',
       'Geo Shape', 'Geo Point'],
      dtype='object')

In [219]:
#Liste des types d'injection exprimés dans l'échantillon
parc['Type Injection'].unique()

array(['Injection totale', 'Injection du surplus',
       'Autoconsommation sans injection'], dtype=object)

In [220]:
#Analyse des types de variables
parc.dtypes

Code Région               int64
Région                   object
Code Département          int64
Nom Département          object
Code Type Prod            int64
Type Production          object
ID_type_inj               int64
Type Injection           object
Tranche                  object
Fin De Trimestre         object
Nombre Installations    float64
Puissance MW            float64
Geo Shape               float64
Geo Point               float64
dtype: object

In [221]:
#Analyse des valeurs manquantes par variable
parc.isna().sum()

Code Région                0
Région                     0
Code Département           0
Nom Département            0
Code Type Prod             0
Type Production            0
ID_type_inj                0
Type Injection             0
Tranche                    0
Fin De Trimestre           0
Nombre Installations     334
Puissance MW             334
Geo Shape               1737
Geo Point               1737
dtype: int64

In [222]:
#Suppression des variables 'Geo Shape' et 'Geo Point' (inutiles dans le contexte traité)
parc.drop(columns=['Geo Shape', 'Geo Point'], inplace=True)

In [223]:
#Quels sont les types de production exploitables dans cet échantillon?
parc['Type Production'].unique()

array(['Photovoltaïque', 'Eolien', 'Hydraulique', 'Bio énergie',
       'Cogénération', 'Autres'], dtype=object)

In [224]:
#Agrégation des données selon de Type de Production
parc.groupby(by='Type Production')['Nombre Installations', 'Puissance MW'].sum().reset_index()

,Type Production,Nombre Installations,Puissance MW
0,Autres,112.0,600.04800
1,Bio énergie,839.0,1151.16235
2,Cogénération,731.0,2461.06100
3,Eolien,1587.0,14539.82261
4,Hydraulique,2015.0,1568.47790
5,Photovoltaïque,450066.0,8610.48292


In [225]:
#Agrégation des données selon le département d'installation
parc.groupby(by=['Nom Département'])['Nombre Installations', 'Puissance MW'].sum()\
.sort_values('Puissance MW', ascending=False).reset_index()

,Nom Département,Nombre Installations,Puissance MW
0,Somme,2376.0,1504.97041
1,Pas-de-Calais,7804.0,1130.89547
2,Aisne,2515.0,1046.77568
3,Marne,3585.0,980.85815
4,Aube,2022.0,868.24973
...,...,...,...
89,Hauts-de-Seine,636.0,50.78203
90,Deux-Sèvres,474.0,46.91844
91,Haute-Saône,2248.0,36.80502
92,Territoire-de-Belfort,1005.0,19.03610


In [226]:
#Agrégation des données selon la Région d'installation
parc.groupby(by=['Région'])['Nombre Installations', 'Puissance MW'].sum()\
.sort_values('Puissance MW', ascending=False).reset_index()

,Région,Nombre Installations,Puissance MW
0,Hauts-de-France,24753.0,4619.842930
1,Grand-Est,31788.0,4366.186716
2,Occitanie,72633.0,3950.842574
3,Nouvelle Aquitaine,60794.0,3082.052850
4,Auvergne-Rhône-Alpes,77598.0,2529.349320
5,Pays de la Loire,48424.0,2019.676240
6,Bretagne,23169.0,1789.707045
7,Provence-Alpes-Côte d'Azur,40606.0,1696.944495
8,Centre-Val de Loire,16198.0,1490.421920
9,Bourgogne-Franche-Comté,24432.0,1317.991870


In [227]:
#Sauvegarde 
parc_production = parc.copy()
parc_production.to_csv('p8_datatable/parc_production.csv')

__`Traitement des données sur la consommation / filières de production 2014 à 2018`__

In [228]:
#Chargement des 4 fichiers rte-puissances-*.csv
import dask.dataframe as dd
rte_puissances = dd.read_csv('p8_data/rte-puissances-*.csv', sep=';', 
                             dtype='float64', parse_dates=[['Date', 'Heures']], encoding='utf-8').compute()

In [229]:
#Aperçu du jeu de données  
rte_puissances

,Date_Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,Solaire,...,Gaz - Cogén.,Gaz - CCG,Gaz - Autres,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Unnamed: 29
0,2014-01-01 00:00:00,64660.0,63200.0,63200.0,454.0,-21.0,2303.0,54144.0,3470.0,0.0,...,1809.0,422.0,72.0,5662.0,2125.0,693.0,460.0,165.0,179.0,NaN
1,2014-01-01 00:15:00,NaN,62650.0,62350.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-01-01 00:30:00,63494.0,62100.0,61500.0,281.0,-6.0,2367.0,54670.0,3356.0,-1.0,...,1847.0,438.0,83.0,5569.0,2450.0,495.0,592.0,170.0,181.0,NaN
3,2014-01-01 00:45:00,NaN,61000.0,60700.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-01-01 01:00:00,61362.0,59900.0,59900.0,281.0,-7.0,2188.0,54000.0,3329.0,-1.0,...,1848.0,258.0,83.0,5457.0,2040.0,76.0,591.0,173.0,181.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,2018-12-31 22:45:00,NaN,62050.0,62550.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35036,2018-12-31 23:00:00,63977.0,63500.0,64000.0,146.0,11.0,3910.0,55589.0,1537.0,-2.0,...,2494.0,1296.0,116.0,3716.0,1432.0,0.0,557.0,318.0,296.0,NaN
35037,2018-12-31 23:15:00,NaN,63750.0,64300.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35038,2018-12-31 23:30:00,63803.0,64000.0,64600.0,146.0,12.0,3452.0,55404.0,1543.0,-2.0,...,2470.0,863.0,115.0,3531.0,1157.0,0.0,557.0,316.0,293.0,NaN


In [230]:
#Changement en Serie temporelle de la variable 'Date_Heures'
rte_puissances['Date_Heures'] = pd.to_datetime(rte_puissances['Date_Heures'])

In [231]:
#Liste des colonnes du jeu de données 
rte_puissances.columns

Index(['Date_Heures', 'Consommation', 'Prévision J-1', 'Prévision J', 'Fioul',
       'Charbon', 'Gaz', 'Nucléaire', 'Eolien', 'Solaire', 'Hydraulique',
       'Pompage', 'Bioénergies', 'Ech. physiques', 'Taux de Co2',
       'Fioul - TAC', 'Fioul - Cogén.', 'Fioul - Autres', 'Gaz - TAC',
       'Gaz - Cogén.', 'Gaz - CCG', 'Gaz - Autres',
       'Hydraulique - Fil de l?eau + éclusée', 'Hydraulique - Lacs',
       'Hydraulique - STEP turbinage', 'Bioénergies - Déchets',
       'Bioénergies - Biomasse', 'Bioénergies - Biogaz', 'Unnamed: 29'],
      dtype='object')

In [232]:
#Suppression des lignes dont la consommation est manquante
#Une ligne sur deux car l'enregistrement temps réel d’électricité est réalisé au pas 30 minutes
rte_puissances.dropna(subset=['Consommation'], axis=0, inplace=True)
rte_puissances.reset_index(drop=True, inplace=True)
display(rte_puissances.shape)
display(rte_puissances.head())

(87648, 29)

,Date_Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,Solaire,...,Gaz - Cogén.,Gaz - CCG,Gaz - Autres,Hydraulique - Fil de l?eau + éclusée,Hydraulique - Lacs,Hydraulique - STEP turbinage,Bioénergies - Déchets,Bioénergies - Biomasse,Bioénergies - Biogaz,Unnamed: 29
0,2014-01-01 00:00:00,64660.0,63200.0,63200.0,454.0,-21.0,2303.0,54144.0,3470.0,0.0,...,1809.0,422.0,72.0,5662.0,2125.0,693.0,460.0,165.0,179.0,NaN
1,2014-01-01 00:30:00,63494.0,62100.0,61500.0,281.0,-6.0,2367.0,54670.0,3356.0,-1.0,...,1847.0,438.0,83.0,5569.0,2450.0,495.0,592.0,170.0,181.0,NaN
2,2014-01-01 01:00:00,61362.0,59900.0,59900.0,281.0,-7.0,2188.0,54000.0,3329.0,-1.0,...,1848.0,258.0,83.0,5457.0,2040.0,76.0,591.0,173.0,181.0,NaN
3,2014-01-01 01:30:00,61217.0,60600.0,60300.0,281.0,-7.0,2190.0,54200.0,3254.0,-1.0,...,1853.0,255.0,83.0,5286.0,1849.0,0.0,595.0,174.0,180.0,NaN
4,2014-01-01 02:00:00,60748.0,59900.0,60200.0,281.0,-6.0,2187.0,53975.0,3245.0,-1.0,...,1853.0,252.0,83.0,5201.0,1592.0,0.0,597.0,174.0,180.0,NaN


In [233]:
#Suppression de la dernière colonne 'Unnamed: 29'
rte_puissances.drop(columns='Unnamed: 29', inplace=True)

In [234]:
#Analyse du type de variables 
rte_puissances.dtypes

Date_Heures                             datetime64[ns]
Consommation                                   float64
Prévision J-1                                  float64
Prévision J                                    float64
Fioul                                          float64
Charbon                                        float64
Gaz                                            float64
Nucléaire                                      float64
Eolien                                         float64
Solaire                                        float64
Hydraulique                                    float64
Pompage                                        float64
Bioénergies                                    float64
Ech. physiques                                 float64
Taux de Co2                                    float64
Fioul - TAC                                    float64
Fioul - Cogén.                                 float64
Fioul - Autres                                 float64
Gaz - TAC 

In [235]:
#Groupement des colonnes par grand type de Puissance
rte_puissances['Fioul'] = rte_puissances['Fioul']\
                        + rte_puissances['Fioul - TAC']\
                        + rte_puissances['Fioul - Cogén.']\
                        + rte_puissances['Fioul - Autres']

rte_puissances['Gaz'] = rte_puissances['Gaz']\
                        + rte_puissances['Gaz - TAC']\
                        + rte_puissances['Gaz - Cogén.']\
                        + rte_puissances['Gaz - CCG']\
                        + rte_puissances['Gaz - Autres']

rte_puissances['Hydraulique'] = rte_puissances['Hydraulique']\
                        + rte_puissances['Hydraulique - Fil de l?eau + éclusée']\
                        + rte_puissances['Hydraulique - Lacs']\
                        + rte_puissances['Hydraulique - STEP turbinage']

rte_puissances['Bioénergies'] = rte_puissances['Bioénergies']\
                        + rte_puissances['Bioénergies - Déchets']\
                        + rte_puissances['Bioénergies - Biomasse']\
                        + rte_puissances['Bioénergies - Biogaz']

#Suppression des colonnes
rte_puissances.drop(columns=['Fioul - TAC', 'Fioul - Cogén.', 'Fioul - Autres', 'Gaz - TAC', 'Gaz - Cogén.', 
                             'Gaz - CCG', 'Gaz - Autres','Hydraulique - Fil de l?eau + éclusée',
                             'Hydraulique - Lacs', 'Hydraulique - STEP turbinage', 'Bioénergies - Déchets', 
                             'Bioénergies - Biomasse', 'Bioénergies - Biogaz', 'Pompage'], inplace=True)

In [236]:
#Analyse des valeurs manquantes
rte_puissances.isna().any()

Date_Heures       False
Consommation      False
Prévision J-1     False
Prévision J       False
Fioul             False
Charbon           False
Gaz               False
Nucléaire         False
Eolien            False
Solaire           False
Hydraulique       False
Bioénergies       False
Ech. physiques    False
Taux de Co2       False
dtype: bool

In [237]:
#Analyse des valeurs doublons
rte_puissances.duplicated().any()

False

In [238]:
#Description rapide avec quelques statistiques descriptives
rte_puissances.describe().T

,count,mean,std,min,25%,50%,75%,max
Consommation,87648.0,54088.793903,11795.658703,29477.0,45138.75,52543.0,62279.50,96272.0
Prévision J-1,87648.0,53798.661669,11714.692868,28500.0,44900.00,52300.0,61800.00,95400.0
Prévision J,87648.0,53734.762402,11699.616191,0.0,45000.00,52200.0,61700.00,96200.0
Fioul,87648.0,558.357612,509.843360,30.0,283.00,448.0,625.00,8455.0
Charbon,87648.0,906.177346,845.285080,-81.0,18.00,713.0,1518.00,4532.0
Gaz,87648.0,6455.486526,4919.808930,469.0,1787.75,5114.0,10003.25,19399.0
Nucléaire,87648.0,45352.607863,6605.128849,27116.0,40414.00,44263.0,50202.25,61712.0
Eolien,87648.0,2539.694893,2005.555317,21.0,1091.00,1914.0,3415.00,12287.0
Solaire,87648.0,934.248870,1368.082871,-2.0,-1.00,14.0,1673.00,6430.0
Hydraulique,87648.0,14132.245607,5583.848061,2774.0,9842.00,13810.0,17984.00,34868.0


In [239]:
rte_puissances

,Date_Heures,Consommation,Prévision J-1,Prévision J,Fioul,Charbon,Gaz,Nucléaire,Eolien,Solaire,Hydraulique,Bioénergies,Ech. physiques,Taux de Co2
0,2014-01-01 00:00:00,64660.0,63200.0,63200.0,907.0,-21.0,4605.0,54144.0,3470.0,0.0,16961.0,1609.0,-4970.0,27.0
1,2014-01-01 00:30:00,63494.0,62100.0,61500.0,562.0,-6.0,4734.0,54670.0,3356.0,-1.0,17028.0,1886.0,-6615.0,27.0
2,2014-01-01 01:00:00,61362.0,59900.0,59900.0,561.0,-7.0,4376.0,54000.0,3329.0,-1.0,15146.0,1889.0,-6317.0,27.0
3,2014-01-01 01:30:00,61217.0,60600.0,60300.0,562.0,-7.0,4380.0,54200.0,3254.0,-1.0,14270.0,1898.0,-5815.0,28.0
4,2014-01-01 02:00:00,60748.0,59900.0,60200.0,562.0,-6.0,4374.0,53975.0,3245.0,-1.0,13586.0,1902.0,-5562.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87643,2018-12-31 21:30:00,60580.0,59300.0,59800.0,320.0,12.0,7650.0,55679.0,1406.0,-2.0,9297.0,2330.0,-3752.0,28.0
87644,2018-12-31 22:00:00,59901.0,58800.0,59400.0,272.0,11.0,7135.0,55404.0,1452.0,-2.0,9265.0,2333.0,-3820.0,26.0
87645,2018-12-31 22:30:00,60939.0,60600.0,61100.0,272.0,11.0,7772.0,55186.0,1569.0,-2.0,8902.0,2330.0,-2660.0,28.0
87646,2018-12-31 23:00:00,63977.0,63500.0,64000.0,272.0,11.0,7820.0,55589.0,1537.0,-2.0,10296.0,2343.0,-2171.0,27.0


In [240]:
#Sauvegarde 
rte_puissances.to_csv('p8_datatable/rte_puissances.csv')

### Calcul de la Puissance appelée pendant un Pic de consommation
L'objectif fixé est de pouvoir estimer un Pic Théorique de Charge, c'est-à-dire dans une mise en situation extrême où l'ensemble du parc électrique et hybride serait en recharge simultanément à la même heure, par exemple en fin de journée à 19h. Quelle serait la charge théorique? Le réseau Enedis pourrait-il tenir ses engagements?

In [241]:
### Création d'un dataframe pour estimer la Puissance appelée pendant un Pic de consommation
pdc_puiss = pd.DataFrame({
    'Année': [2019, 2021, 2030, 2035, 2040],
    'Connecteur_puiss_kW': [22, 22, 22, 22, 22], #Voir l'analyse des puissances, 22kW représente 50% des connecteurs installés
    'nbre_ve': [immat_motor_annee['Electricité'].max(), ve2021, 10*1e6, 15*1e6, 20*1e6], #Immat. Max (Cumul jusqu'à fin 2019)
    'nbre_vh': [immat_motor_annee['Hybride'].max(), vh2021, 0, 0, 0],     #Immat. Max (Cumul jusqu'à fin 2019)
    'fréquence_utilisation': [1, 1, 1, 1, 1] #Une seule charge, l'hypothèse d'une charge globale à 19H par exemple
})

pdc_puiss['Puissance_kW'] = pdc_puiss['Connecteur_puiss_kW'] * (pdc_puiss['nbre_ve'] + pdc_puiss['nbre_vh'])
pdc_puiss['Puissance_MW'] = pdc_puiss['Puissance_kW']/1e3
pdc_puiss['Puissance_GW'] = pdc_puiss['Puissance_kW']/1e6
pdc_puiss

,Année,Connecteur_puiss_kW,nbre_ve,nbre_vh,fréquence_utilisation,Puissance_kW,Puissance_MW,Puissance_GW
0,2019,22,164970.0,549381,1,15715722.0,15715.722,15.715722
1,2021,22,247518.0,814890,1,23372976.0,23372.976,23.372976
2,2030,22,10000000.0,0,1,220000000.0,220000.000,220.000000
3,2035,22,15000000.0,0,1,330000000.0,330000.000,330.000000
4,2040,22,20000000.0,0,1,440000000.0,440000.000,440.000000


In [242]:
#Sauvegarde
pdc_puiss.to_csv('p8_datatable/pdc_puiss.csv')

A fin de 2019, l'ensemble des électriques + hybrides en recharge à la même heure nécessiteraient un appel de charge d'une Puissance approximative de 15,7 GW, et un pic de 23,3 GW à fin 2021. 

A titre de comparaison, le plus grand Pic de consommation enregistré entre 2014 et 2018 (voir stat. descriptives ci-dessus) était de 96272 MW, soit 96 GW. 

Notons, que dans le futur, la consommation globale de notre Pays devrait baisser. Le chauffage sera optimisé par des pompes, des éclairages LED seront davantage généralisés, de plus en plus de toits photovoltaïques, le poids des véhicules et les frottements seront réduits, etc… Tout cela contribuera à abaisser la consommation. Reste à savoir combien de véhicules électriques circuleront en 2040 pour évaluer les besoins en électricité. Là aussi, des facteurs technologiques, des décisions politiques, des changements dans nos modes de vie, etc… rendent la question complexe. Mais restons sur nos modes de vie connus à ce jour pour estimer globalement la consommation moyenne sur une année complète.

### Calcul de la consommation annuelle moyenne des Véhicules propres
Actuellement, une voiture électrique consomme en moyenne 20 kWh pour 100 Km (voir calcul fin Mission 1). Selon l’INSEE, les voitures particulières ont parcouru 13 274 km en moyenne en 2016. En supposant que nous resterions à ce niveau de kilométrage, une voiture électrique consommerait ainsi 2 650 kWh par an en moyenne.

In [243]:
### Création d'un dataframe pour estimer la consommation moyenne / an
conso_an = pd.DataFrame({
    'Année': [2019, 2021, 2030, 2035, 2040],
    'nbre_ve': [(immat_motor_annee['Electricité'].max() + immat_motor_annee['Hybride'].max()), 
                (ve2021+vh2021), 10*1e6, 15*1e6, 20*1e6],
    'conso_kWh/km':[20, 20, 20, 20, 20],
    'distance_km/an':[13274, 13274, 13274*.9, 13274*.8, 13274*.8]
})

conso_an['conso_moy_kWh'] = conso_an['nbre_ve'] * conso_an['conso_kWh/km']/100 * conso_an['distance_km/an']
conso_an['conso_moy_TWh'] = round(conso_an['conso_moy_kWh']*1e-9, 2)
    
conso_an    

,Année,nbre_ve,conso_kWh/km,distance_km/an,conso_moy_kWh,conso_moy_TWh
0,2019,714351.0,20,13274.0,1.896459e+09,1.90
1,2021,1062408.0,20,13274.0,2.820481e+09,2.82
2,2030,10000000.0,20,11946.6,2.389320e+10,23.89
3,2035,15000000.0,20,10619.2,3.185760e+10,31.86
4,2040,20000000.0,20,10619.2,4.247680e+10,42.48


In [244]:
#Sauvegarde
conso_an.to_csv('p8_datatable/conso_an.csv')

**Dans tous les cas, la consommation des véhicules sera inférieure à celle des radiateurs électriques actuelles qui représente des 158,5 TWh de la consommation résidentielle (source déclaration RTE).**

Rappelons également que notre Pays s'est engagé dans un processus de transition énergétique, il ne s'agit donc pas d'additionner les consommations futures à celles d'aujourd'hui. Or, d’importantes économies d’énergie vont être réalisées grâce à l’isolation des logements, la généralisation d’appareils électroménagers moins énergivores ou encore l’élimination des radiateurs électriques au profit de systèmes plus performants (pompe à chaleur) ou non-électriques. Sachant aussi que la consommation des véhicules électriques baissera, les technologies sont en évolution permanente! 

[Retour vers la page notebook précédente traitant du maillage en IRVE France 2020 et prévision à 5 ans](https://github.com/nalron/project_electric_cars_france2040/blob/french_version/p8_notebook03.ipynb)